# 1. Library Define

In [25]:
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd
from datetime import datetime, date, time

# 2. MNIST Data Import

In [6]:
# MNIST 데이터 임포트
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


# 3. Parameter Define

In [8]:
# Define some parameters
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128


# 텐서보드 모델 요약을 저장할 위치
LOG_DIR = "logs/RNN_with_summaries_"+datetime.now().strftime('%Y%m%d%H%M%S')+"/"


# 4. Create placeholders for inputs, labels

In [9]:
# Create placeholders for inputs, labels
_inputs = tf.placeholder(tf.float32,
                         shape=[None, time_steps, element_size],
                         name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name='labels')
_inputs.get_shape(), y.get_shape()

(TensorShape([Dimension(None), Dimension(28), Dimension(28)]),
 TensorShape([Dimension(None), Dimension(10)]))

# 5. Tensorflow에서 로깅 요약을 추가하는 작업

In [10]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

# 6. RNN작업 구성

In [11]:
# RNN 모델 계산
def rnn_step(previous_hidden_state, x):
    current_hidden_state = tf.tanh(tf.matmul(previous_hidden_state, Wh) + tf.matmul(x, Wx) + b_rnn)
    
    return current_hidden_state

In [12]:
# Weight X, Weights h, bias Variable Define
with tf.name_scope('rnn_weights'):
    with tf.name_scope("W_x"):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope("W_h"):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope("Bias"):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

# 7. Input으로 RNN 실행 그래프 정의

In [13]:
# _input shape : (batch_size, time_step, element)
# processed_input shape : (time_steps, batch_size, element_size)
processed_input = tf.transpose(_inputs, perm=[1, 0, 2])

# hidden_layer output tensor define
initial_hidden = tf.zeros([batch_size, hidden_layer_size])

# Getting all state vectors across time
# all_hidden_states shape : (time_step, batch_size, hidden_layer_size)
all_hidden_states = tf.scan(rnn_step,
                            processed_input,
                            initializer=initial_hidden,
                            name='states')


# 8. hidden_layer output으로 각 클래스별 score 계산

In [14]:
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bl

In [15]:
# Weights for output layers
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0, stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope("Bias_linear"):
        bl = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=.01))
        variable_summaries(bl)

In [18]:
with tf.name_scope('linear_layer_weights') as scope:
    # Iterate across time, apply linear layer to all RNN outputs
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    output = all_outputs[-1]     # Get Last output
    tf.summary.histogram('outputs', output)
all_outputs.get_shape(), output.get_shape()  # 앞쪽은 모든 output, 두번째는 마지막 output(최종)

(TensorShape([Dimension(28), Dimension(128), Dimension(10)]),
 TensorShape([Dimension(128), Dimension(10)]))

# 9. Loss Function / Train / Accuracy Define

In [26]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    # Using RMSPropOptimizer
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(output, 1))

    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar('accuracy', accuracy)
    
# Merge all the summaries
merged = tf.summary.merge_all()

# 10. Training Session Run

In [24]:
# Get a small test set
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

with tf.Session() as sess:
    # Write summaries to LOG_DIR -- used by TensorBoard
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train', graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test', graph=tf.get_default_graph())

    sess.run(tf.global_variables_initializer())
    for i in range(2000):

            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Reshape data to get 28 sequences of 28 pixels
            batch_x = batch_x.reshape((batch_size, time_steps, element_size))
            summary, _ = sess.run([merged, train_step], feed_dict={_inputs: batch_x, y: batch_y})
            # Add to summaries
            train_writer.add_summary(summary, i)

            if i == 0 or (i+1) % 1000 == 0:
                acc, loss, = sess.run([accuracy, cross_entropy], feed_dict={_inputs: batch_x, y: batch_y})
                print("Iter " + str(i+1) + ", Minibatch Loss= " +
                      "{:.6f}".format(loss) + ", Training Accuracy= " +
                      "{:.5f}".format(acc))
            if (i+1) % 100 == 0:
                # Calculate accuracy for 128 mnist test images and
                # add to summaries
                summary, acc = sess.run([merged, accuracy], feed_dict={_inputs: test_data, y: test_label})
                test_writer.add_summary(summary, i+1)
                test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,y: test_label})
                print("Test Iter {} Accuracy: {}".format(i+1, test_acc))
print("Complete")

Iter 1, Minibatch Loss= 2.303251, Training Accuracy= 9.37500
Test Iter 100 Accuracy: 11.71875
Test Iter 200 Accuracy: 16.40625
Test Iter 300 Accuracy: 27.34375
Test Iter 400 Accuracy: 35.9375
Test Iter 500 Accuracy: 30.46875
Test Iter 600 Accuracy: 53.90625
Test Iter 700 Accuracy: 55.46875
Test Iter 800 Accuracy: 57.8125
Test Iter 900 Accuracy: 58.59375
Iter 1000, Minibatch Loss= 1.211384, Training Accuracy= 55.46875
Test Iter 1000 Accuracy: 60.9375
Test Iter 1100 Accuracy: 64.84375
Test Iter 1200 Accuracy: 64.84375
Test Iter 1300 Accuracy: 65.625
Test Iter 1400 Accuracy: 66.40625
Test Iter 1500 Accuracy: 68.75
Test Iter 1600 Accuracy: 78.90625
Test Iter 1700 Accuracy: 76.5625
Test Iter 1800 Accuracy: 80.46875
Test Iter 1900 Accuracy: 84.375
Iter 2000, Minibatch Loss= 0.538656, Training Accuracy= 82.81250
Test Iter 2000 Accuracy: 87.5
Complete
